## Import libraries to get the env variables

In [8]:
import os
from dotenv import load_dotenv

In [9]:
load_dotenv()

OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE")
OPENAI_API_VERSION = "2023-03-15-preview"

## Import the libraries to get the documents from txt file

In [10]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

## Get the documents from txt file

In [11]:
loader = TextLoader("text.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

docs

[Document(page_content='The Volvo S90: Where Elegance Meets Innovation, The Volvo S90, a flagship luxury sedan from the Swedish automaker, is a testament to the brand\'s commitment to safety, performance, and Scandinavian design principles. Introduced in 2016, the S90 immediately made a strong impression in the luxury sedan segment and has continued to evolve with each subsequent model year. Design and Aesthetics: At first glance, the Volvo S90 captivates with its sleek and understated design. Clean lines, a bold grille adorned with the iconic Volvo emblem, and distinctive Thor\'s Hammer LED headlights create an unmistakable presence on the road. The exterior design is a fusion of timeless elegance and modern sophistication, setting it apart from more ostentatious luxury cars. Step inside, and you\'re greeted with an interior that embodies Scandinavian luxury. Premium materials such as Nappa leather, real wood trim, and brushed metal accents adorn the cabin. The minimalist design philo

## Import libraries for embeddings and store

In [12]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [13]:
embedding = OpenAIEmbeddings()

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
query = "From what country is Volvo?"

In [16]:
db2 = Chroma.from_documents(docs, embedding_function, persist_directory="databse")
docs = db2.similarity_search(query)

In [17]:
db3 = Chroma(persist_directory="databse", embedding_function=embedding_function)
docs1 = db3.similarity_search(query)
docs1

[Document(page_content='The Volvo S90: Where Elegance Meets Innovation, The Volvo S90, a flagship luxury sedan from the Swedish automaker, is a testament to the brand\'s commitment to safety, performance, and Scandinavian design principles. Introduced in 2016, the S90 immediately made a strong impression in the luxury sedan segment and has continued to evolve with each subsequent model year. Design and Aesthetics: At first glance, the Volvo S90 captivates with its sleek and understated design. Clean lines, a bold grille adorned with the iconic Volvo emblem, and distinctive Thor\'s Hammer LED headlights create an unmistakable presence on the road. The exterior design is a fusion of timeless elegance and modern sophistication, setting it apart from more ostentatious luxury cars. Step inside, and you\'re greeted with an interior that embodies Scandinavian luxury. Premium materials such as Nappa leather, real wood trim, and brushed metal accents adorn the cabin. The minimalist design philo

In [20]:
retriever = db3.as_retriever()
docs2 = retriever.get_relevant_documents("From what country is volvo")

len(docs2)
docs2

[Document(page_content='The Volvo S90: Where Elegance Meets Innovation, The Volvo S90, a flagship luxury sedan from the Swedish automaker, is a testament to the brand\'s commitment to safety, performance, and Scandinavian design principles. Introduced in 2016, the S90 immediately made a strong impression in the luxury sedan segment and has continued to evolve with each subsequent model year. Design and Aesthetics: At first glance, the Volvo S90 captivates with its sleek and understated design. Clean lines, a bold grille adorned with the iconic Volvo emblem, and distinctive Thor\'s Hammer LED headlights create an unmistakable presence on the road. The exterior design is a fusion of timeless elegance and modern sophistication, setting it apart from more ostentatious luxury cars. Step inside, and you\'re greeted with an interior that embodies Scandinavian luxury. Premium materials such as Nappa leather, real wood trim, and brushed metal accents adorn the cabin. The minimalist design philo

## Import the model 3.5 turbo

In [21]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA

llm = AzureChatOpenAI(
    deployment_name = "gpt35-uif54579",
    model_name = "gpt-35-turbo",
    temperature = 0.7
)

In [22]:
llm

AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.7, model_kwargs={}, openai_api_key='9ac347d13e834f288a2076ff9c7b418a', openai_api_base='https://sbzdfopenai.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None, deployment_name='gpt35-uif54579', openai_api_type='azure', openai_api_version='2023-03-15-preview')

## Create a chain and use the query in the model

In [23]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever = retriever)

In [24]:
query1 = 'From what country is volvo'
llm_response = chain(query1)
llm_response

{'query': 'From what country is volvo',
 'result': 'Volvo is a company based in Sweden.'}

In [25]:
llm_response['result']

'Volvo is a company based in Sweden.'